Svm is a binary classifier, the algorithm will need either use one vs one or one vs all for multiclassification. The strategy for multi-classification can be set with `decision function shape` when creating an object for the SVC class. For one vs one set `decision_function_shape` equal to `'ovo'`, while for one vs all use `'ovr'` implement both strategies for a multi-class data set and observe which strategy is faster and more accurate  

Use `SVR` for the carlifornia housing data; you should experiment with using different kernels. What kernel works best?

Support vector regression (with a linear kernel) and linear regression are similar but use a different cost function; given the loss functions which model would you think would work better with the presence of outliers? Test out your answers by using both support vector regression and linear regression to fit a linear through a data set using a outlier

In [1]:
#I cant do question one because i havent mastered label and one hot encoding
#Question 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import svm

In [2]:
#Getting the dataset
data = fetch_california_housing()
X = data['data']
y = data['target']
data['feature_names'].append('MedianHomePrices')
#Visualising it in a pandas dataframe
df = pd.DataFrame(np.hstack((X,y.reshape(-1,1))), columns = data['feature_names'])
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHomePrices
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [3]:
scaler = StandardScaler()
rgr = svm.SVR(kernel='rbf')
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
from tempfile import mkdtemp
from shutil import rmtree
cache_dir = mkdtemp()
#Creating a pipeline for the workflow
pipe = Pipeline([("scaler",scaler),("rgr",rgr)],memory = cache_dir)
#Tuning hyperparameters
#using Svm with the radial basis function kernel 
param_grid = {"rgr__C": np.logspace(-2,2,10),
             "rgr__gamma":np.logspace(-2,2,10)}
randomized_search = RandomizedSearchCV(pipe,param_grid,cv=5,n_jobs=2,n_iter=50,verbose=1)
randomized_search.fit(X_train,y_train)

# Doing the same thing using kernel approximations because it is faster

In [9]:
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.kernel_approximation import Nystroem
#Nystroem?
Kernel_approx = Nystroem(kernel='rbf', n_components=100, random_state=0)
rgr = svm.LinearSVR()

cachedir = mkdtemp()
pipe = Pipeline([("scaler",scaler),("projection",Kernel_approx),("rgr",rgr)],memory=cachedir)
param_grid = {"projection__gamma":np.logspace(-2,2,100),
             "rgr__C":np.logspace(-2,2,100)}
random_search = RandomizedSearchCV(pipe,param_grid,cv=5,n_jobs=2,n_iter=50,verbose=1,random_state=0)
random_search.fit(X_train, y_train)
rmtree(cachedir)
#.best_score_ returns the r2_value of the crossvalidation set
#check out rbf sampler

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   56.7s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  5.3min
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:  6.7min finished
C:\Users\HP ELITEBOOK\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [12]:
random_search.score(X_train,y_train)

0.7305254966900034